# Importants!

In [6]:
import pandas as pd
from scipy.signal import butter,filtfilt
from scipy.signal import find_peaks
import numpy as np
import plotly.graph_objects as go

# Convert data to pandas

In [25]:
import pandas as pd
df = pd.read_csv('uoftdb.csv')
data = {'id':[], 'day':[], 'pos':[],'signal':[]}
for elem in df:
    params = elem.split('_')
    data['id'].append(params[0])
    data['day'].append(params[1])
    data['pos'].append(params[2])
    data['signal'].append(df[elem].to_list())
df_new = pd.DataFrame.from_dict(data)
df_sitted = df_new[df_new['pos'] == "sit"]

# Filter function

In [7]:
def butter_lowpass_filter(data, cutoff, fs, order):
    nyq = fs /2
    normal_cutoff = cutoff / nyq
    # Get the filter coefficients 
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    y = filtfilt(b, a, data)
    return y
    
def butter_bandpass_filter(data, cutoff, fs, order):
    nyq = fs/2
    normal_cutoff = [f/nyq for f in cutoff]
    # Get the filter coefficients 
    b, a = butter(order, normal_cutoff, btype='bandpass', analog=False)
    y = filtfilt(b, a, data)
    return y

# Filtering one sample

In [8]:
# Filter Arguments:
T = 10.0                 # Sample Period
fs = 200.0              # sample rate, Hz
cutoff_BPF = [0.5, 40]  # desired cutoff frequency of the filter, Hz
cutoff_LPF = 40
order = 6       
n = int(T * fs)         # total number of samples
start_point = 1000      # where to start original signal cutting

# The data:
all_data = df['1_1_sit'].to_numpy()
sampled_data = all_data[start_point : start_point+n+1]

filttered_BPF = butter_bandpass_filter(sampled_data, cutoff_BPF, fs, order)
filttered_LPF = butter_lowpass_filter(sampled_data, cutoff_LPF, fs, order)

fig = go.Figure()
fig.add_trace(go.Scatter(
            y = sampled_data,
            line =  dict(shape =  'spline' ),
            name = 'Original noisy signal'
            ))
fig.add_trace(go.Scatter(
            y = filttered_BPF,
            line =  dict(shape =  'spline' ),
            name = 'filtered signal with BPF ' + str(cutoff_BPF)
            ))
fig.add_trace(go.Scatter(
            y = filttered_LPF,
            line =  dict(shape =  'spline' ),
            name = 'filtered signal with LPF ' + str(cutoff_LPF)
            ))
fig.show()

# Filter all sitted signal

In [29]:
df_sitted['signal'].apply(lambda sig: butter_bandpass_filter(sig, cutoff_BPF, fs, order))

0       [0.17415526488561872, 0.2544510642087944, 0.28...
1       [-0.005889644447419976, 0.00751706139049913, 0...
2       [0.13988075187055268, 0.1831824102441966, 0.17...
3       [0.24018141872158275, 0.5433873636443802, 0.67...
4       [0.021469642977226252, 0.001127648143815345, -...
                              ...                        
1614    [-0.09008956747937615, -0.10658470297840655, -...
1617    [-0.026784870718831462, -0.02688902567396126, ...
1621    [0.0009926999161081582, -0.008779096322349711,...
1623    [0.04441367741370629, 0.017497162919676085, -0...
1625    [0.0004967294026147838, -0.010052926620838601,...
Name: signal, Length: 1288, dtype: object

In [30]:
fig = go.Figure()
fig.add_trace(go.Scatter(
            y = df_sitted.loc[0]['signal'],
            line =  dict(shape =  'spline' ),
            name = 'Original noisy signal'
            ))
fig.show()

# Finding picks

In [47]:
minimal_peak_value = 0.2
peaks, _ = find_peaks(filttered_BPF, height=minimal_peak_value)


#print(peaks)
fig = go.Figure()
fig.add_trace(go.Scatter(
            y = filttered_BPF,
            line =  dict(shape =  'spline' ),
            name = 'Signal'
            ))
fig.add_trace(go.Scatter(
            x = peaks,
            y = filttered_BPF[peaks],
            name = 'paeks'
            ))
fig.show()

[   2    7   16   37  143  199  357  531  712  878 1039 1216 1390 1547
 1727 1769 1921]
